```{contents}
```
## Async Node

An **Async Node** in LangGraph is a node whose execution function is declared as `async` and executed within LangGraph’s asynchronous runtime.
It enables **non-blocking execution**, **parallel I/O**, and **high-throughput pipelines**, which are essential for production-grade LLM systems.

---

### **1. Motivation: Why Async Nodes Exist**

LLM workflows are dominated by **I/O-bound operations**:

| Operation        | Nature      |
| ---------------- | ----------- |
| LLM calls        | Network I/O |
| Tool APIs        | Network I/O |
| Database queries | I/O         |
| Vector search    | I/O         |
| File systems     | I/O         |

Synchronous execution blocks the runtime and wastes resources.
**Async nodes allow concurrency without threads.**

---

### **2. Conceptual Model**

**Synchronous Node**

```
Node A → wait → Node B → wait → Node C
```

**Async Node**

```
Node A ─┬─→ await
        ├─→ await
        └─→ await
```

Multiple operations proceed concurrently under one event loop.

---

### **3. Defining an Async Node**

An async node is defined using `async def`.

```python
async def fetch_data(state):
    result = await external_api_call(state["query"])
    return {"data": result}
```

```python
builder.add_node("fetch", fetch_data)
```

LangGraph automatically detects coroutine functions and schedules them asynchronously.

---

### **4. Minimal Working Example**

```python
from langgraph.graph import StateGraph, END
from typing import TypedDict
import asyncio

class State(TypedDict):
    value: int

async def async_increment(state):
    await asyncio.sleep(1)
    return {"value": state["value"] + 1}

builder = StateGraph(State)
builder.add_node("async_inc", async_increment)
builder.set_entry_point("async_inc")
builder.add_edge("async_inc", END)

graph = builder.compile()

result = asyncio.run(graph.ainvoke({"value": 1}))
print(result)
```

**Behavior**

* Non-blocking delay
* Event loop remains responsive
* High scalability

---

### **5. Parallel Async Execution**

Multiple async nodes can run concurrently.

```python
async def call_api_a(state):
    return {"a": await api_a()}

async def call_api_b(state):
    return {"b": await api_b()}
```

```python
builder.add_node("A", call_api_a)
builder.add_node("B", call_api_b)

builder.add_edge("start", "A")
builder.add_edge("start", "B")
builder.add_edge("A", "join")
builder.add_edge("B", "join")
```

Both API calls execute in parallel.

---

### **6. State Semantics in Async Nodes**

LangGraph enforces **deterministic state updates**:

| Rule            | Meaning                                |
| --------------- | -------------------------------------- |
| Isolation       | Each node receives a snapshot of state |
| Partial updates | Node returns only modified fields      |
| Reducer merge   | Concurrent updates resolved by reducer |

This prevents race conditions even under concurrency.

---

### **7. Async vs Sync Nodes**

| Feature        | Sync Node | Async Node   |
| -------------- | --------- | ------------ |
| Execution      | Blocking  | Non-blocking |
| Concurrency    | Limited   | High         |
| I/O efficiency | Low       | High         |
| Scalability    | Moderate  | Excellent    |

---

### **8. When to Use Async Nodes**

Use async nodes when:

* Calling LLM APIs
* Using vector databases
* Calling microservices
* Performing file I/O
* Running parallel tools

Avoid async for pure CPU-heavy computation.

---

### **9. Production Best Practices**

| Practice          | Benefit               |
| ----------------- | --------------------- |
| Use `ainvoke`     | Enables async runtime |
| Limit concurrency | Prevent overload      |
| Add timeouts      | Avoid hangs           |
| Use retries       | Fault tolerance       |
| Use semaphores    | Resource control      |

---

### **10. Mental Model**

Async nodes transform LangGraph from a **pipeline executor** into an **event-driven workflow engine** capable of serving production-scale LLM systems.


### Demonstration

In [1]:
from typing import TypedDict
import asyncio

class State(TypedDict):
    query: str
    llm_result: str
    db_result: str


async def fake_llm_call(query: str):
    await asyncio.sleep(2)  # simulate network latency
    return f"LLM answer for: {query}"

async def fake_db_search(query: str):
    await asyncio.sleep(2)
    return f"DB result for: {query}"

async def llm_node(state: State):
    result = await fake_llm_call(state["query"])
    return {"llm_result": result}

async def db_node(state: State):
    result = await fake_db_search(state["query"])
    return {"db_result": result}

In [5]:
from langgraph.graph import StateGraph, END

builder = StateGraph(State)

builder.add_node("llm", llm_node)
builder.add_node("db", db_node)

builder.set_entry_point("start")

builder.add_node("start", lambda state: state)

builder.add_edge("start", "llm")
builder.add_edge("start", "db")

builder.add_edge("llm", END)
builder.add_edge("db", END)

graph = builder.compile()

In [6]:
import time

start_time = time.time()

result = await graph.ainvoke({
    "query": "LangGraph async nodes"
})

print(result)
print("Elapsed:", time.time() - start_time)


{'query': 'LangGraph async nodes', 'llm_result': 'LLM answer for: LangGraph async nodes', 'db_result': 'DB result for: LangGraph async nodes'}
Elapsed: 2.016120433807373
